In [ ]:
import torch
import transformers
import pandas as pd
import numpy as np
import wandb
import evaluate
from argparse import Namespace
from tqdm.auto import tqdm
from datasets import load_from_disk
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DataCollatorWithPadding, get_scheduler, AdamW
from torch.nn.functional import cross_entropy
from torch.utils.data import DataLoader

### Load the dataset

In [ ]:
ds = load_from_disk("../../Violence_data/geo_corpus.0.0.1_dataset_for_train")

In [ ]:
ds

In [ ]:
ds["train"][0]

# Full training with native Pytorch and DataLoader

This code was inspired from the Transformers course available in Huggingface (Chapter 3: A full training)

### Setup the hyperparameters and other variables for training and wrap them in a *Namespace* for easy access

In [ ]:
config = {
    "model_ckpt": "setu4993/smaller-LaBSE",
    "batch_size": 1024,
    "num_labels" : 6,
    "init_lr": 5e-5,
    "num_epochs": 3,
    "num_warmup_steps": 0,
    "cuda_device": "cuda:12",
    "lr_scheduler_type": "cosine", # linear
    "weight_decay": 0.1,
    "max_length": 32,
    "seed": 42
}

args = Namespace(**config)

### From text to tokens

In [ ]:
# Instantiate the tokenizer
model_ckpt = args.model_ckpt
tokenizer = BertTokenizerFast.from_pretrained(model_ckpt, 
                                              model_max_length=args.max_length)

### Tokenizing the whole dataset

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

In [ ]:
%time tokenized_ds = ds.map(tokenize, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Prepare for training

In [ ]:
tokenized_ds

In [ ]:
# Remove column (text) and leave the columns the model expect for training
tokenized_ds = tokenized_ds.remove_columns('text')
tokenized_ds.set_format("torch")
tokenized_ds["train"].column_names

In [ ]:
tokenized_ds["train"][0]

In [ ]:
tokenized_ds["train"].features

### Define the dataloaders

In [ ]:
# Select a subsample for testing purposes
train_tokenized_ds = tokenized_ds["train"].select(range(20000))
validation_tokenized_ds = tokenized_ds["validation"].select(range(5000))

In [ ]:
#train_dataloader = DataLoader(tokenized_ds["train"], shuffle=True, 
#                               batch_size=args.batch_size, collate_fn=data_collator)
# eval_dataloader = DataLoader(tokenized_ds["validation"],
#                             batch_size=args.batch_size, collate_fn=data_collator)
train_dataloader = DataLoader(train_tokenized_ds, shuffle=True, 
                              batch_size=args.batch_size, collate_fn=data_collator)
eval_dataloader = DataLoader(validation_tokenized_ds,
                            batch_size=args.batch_size, collate_fn=data_collator)

In [ ]:
# Inspect a batch to check if there are no mistakes
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

### Define some helper functions

In [ ]:
# Differentiate the parameters that should receive weight decay (Biases and LayerNorm weights
# are not subject to weight decay)
def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [{'params': params_with_wd, 'weight_decay': args.weight_decay},
           {'params': params_without_wd, 'weight_decay': 0.0}]
                

### Instantiate the model, define optimizer and learning rate scheduler

In [ ]:
id2label = {0: 'post7geo10', 1: 'post7geo30', 2: 'post7geo50',
           3: 'pre7geo10', 4: 'pre7geo30', 5: 'pre7geo50'}
label2id = {'post7geo10': 0, 'post7geo30': 1, 'post7geo50': 2,
            'pre7geo10': 3, 'pre7geo30': 4, 'pre7geo50': 5}

In [ ]:
# Instantiate the model
model = BertForSequenceClassification.from_pretrained(args.model_ckpt, 
                                                      num_labels = args.num_labels,
                                                     problem_type = "multi_label_classification",
                                                     id2label = id2label,
                                                     label2id = label2id)

In [ ]:
# A test to make sure we have everything working properly when we pass our batch to this model
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
# Define optimizer
optimizer = AdamW(get_grouped_params(model), lr=args.init_lr)

In [ ]:
# Define the learning rate scheduler
num_epochs = args.num_epochs
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name = args.lr_scheduler_type,
    optimizer = optimizer,
    num_warmup_steps = args.num_warmup_steps,
    num_training_steps = num_training_steps
)
print(num_training_steps)

In [ ]:
def get_lr():
    return optimizer.param_groups[0]['lr']

### The training loop

In [ ]:
device = torch.device(args.cuda_device) if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

### Evaluation Loop

In [ ]:
progress_bar = tqdm(range(len(eval_dataloader)))

accuracy_metric = evaluate.load("accuracy")
roc_auc_metric = evaluate.load("roc_auc", "multilabel")
precision_micro_metric = evaluate.load("precision")
precision_weighted_metric = evaluate.load("precision")
recall_micro_metric = evaluate.load("recall")
recall_weighted_metric = evaluate.load("recall")
f1_micro_metric = evaluate.load("f1")
f1_weighted_metric = evaluate.load("f1")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    pred_prob = torch.sigmoid(outputs.logits)
    preds = (pred_prob > 0.5)*1
        
    for references, predictions in zip (batch["labels"], preds):
        accuracy_metric.add_batch(predictions=predictions, 
                              references=references)
        roc_auc_metric.add_batch(prediction_scores=preds, 
                              references=batch["labels"])
        precision_micro_metric.add_batch(predictions=predictions, 
                              references=references)
        precision_weighted_metric.add_batch(predictions=predictions, 
                              references=references)
        recall_micro_metric.add_batch(predictions=predictions, 
                              references=references)
        recall_weighted_metric.add_batch(predictions=predictions, 
                              references=references)
        f1_micro_metric.add_batch(predictions=predictions, 
                              references=references)
        f1_weighted_metric.add_batch(predictions=predictions, 
                              references=references)
    progress_bar.update(1)
    
accuracy_res = accuracy_metric.compute()
%time roc_auc_res = roc_auc_metric.compute(average="micro")
precision_micro_res = precision_micro_metric.compute(average="micro")
precision_weighted_res = precision_weighted_metric.compute(average="weighted")
recall_micro_res = recall_micro_metric.compute(average="micro")
recall_weighted_res = recall_weighted_metric.compute(average="weighted")
f1_micro_res = f1_micro_metric.compute(average="micro")
f1_weighted_res = f1_weighted_metric.compute(average="weighted")

In [ ]:
print("Accuracy: ", accuracy_res)
print("roc_auc: ", roc_auc_res)
print("Precision_micro: ", precision_micro_res)
print("Precision_weighted: ", precision_weighted_res)
print("Recall_micro: ", recall_micro_res)
print("Recall_weighted: ", recall_weighted_res)
print("F1_micro: ", f1_micro_res)
print("F1_weighted: ", f1_weighted_res)